# DD2367 - Programming Assignment II: Quantum Circuits and Gates - Bell's Circuit and SWAP Test #

Authors: Scott McHaffie, Jai Iyer, Venkatesh Elayaraja

# Task 1 - State‑Vector Quantum Computer Simulator  #

## 1.1 - State‑vector representation ##

## 1.2 - Quantum gate implementation ##

## 1.3 - Circuit simulation: Bell state ##

## 1.4 - Output, verification, and circle‑plot comparison ##

# Task 2 - State‑Vector SWAP Test  #

# Bonus Tasks - Quantum State Comparison with the Multiqubit SWAP Test #

## B.1 - Basis vs $|5\rangle$: design, simulate, evaluate ##

## B.2 - Introducing superposition ##

# Generative AI Disclosure #

We used ChatGPT-5 and ChatGPT-4 for AI assistance during this assignment. A breakdown of the usage per task is shown below:

1. Task 1: 

2. Task 2: 

3. Bonus: 